# Projeto: Avaliando Modelo de Decision Tree para Previsão de Derrames

                    
* Dataset: stroke_data.csv obtido no [Kaggle](https://www.kaggle.com/fedesoriano/stroke-prediction-dataset).

> Neste projeto, realizaremos uma breve análise exploratória dos dados, culminando na construção e avaliação de um modelo de Árvore de Decisão utilizando PySpark. O objetivo é prever casos de derrame (stroke) com base nos dados disponíveis.

> Este projeto faz parte do módulo de Apache Spark da Pós-Graduação em Ciência de Dados da XP Educação

In [3]:
# Vamos criar o objeto sparkSession
from pyspark.sql import SparkSession

# Spark entry point
spark = SparkSession \
.builder \
.appName("Decision Tree") \
.getOrCreate()

spark.version

'3.5.0'

In [4]:
# Vamos ler o arquivo com os dados de eventos de derrame
df = spark.read.csv('D:/Abf/7 - Pos_XP_Educacao/Cientista de Dados/Módulo 2/Desafio/stroke_data.csv',header=True, inferSchema='True')


In [4]:
# Verificando as primeiras 5 entradas
df.show(5)

+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
|  0|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level|  bmi| smoking_status|stroke|
+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
|  1|Female|18.0|           0|            0|          No|      Private|         Urban|            94.19|12.12|         smokes|     1|
|  2|  Male|58.0|           1|            0|         Yes|      Private|         Rural|           154.24| 33.7|   never_smoked|     0|
|  3|Female|36.0|           0|            0|         Yes|     Govt_job|         Urban|            72.63| 24.7|         smokes|     0|
|  4|Female|62.0|           0|            0|         Yes|Self-employed|         Rural|            85.52| 31.2|formerly smoked|     0|
|  5|Female|82.0|           0|            0|         Yes|     

#### 1) Quantos registros (linhas) existem no arquivo?

In [4]:
# Contagem de linhas
df.count()

67135

#### 2) Quantas colunas existem no arquivo? Quantas são numéricas?
###### R: 12 colunas e 7 são numéricas

In [15]:
# Verificando os tipos de dados do Dataframe. 
df.printSchema()

root
 |-- 0: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



#### 3) No conjunto de dados, quantos pacientes sofreram e não sofreram derrame (stroke), respectivamente?

In [17]:
# Agrupando a contgem dos valores da coluna stroke 0 (não) e 1(sim)

df.groupBy("stroke").count().show()

+------+-----+
|stroke|count|
+------+-----+
|     1|40287|
|     0|26848|
+------+-----+



#### 4) Quantos pacientes sofreram derrame e trabalhavam respectivamente, no setor privado, de forma independente, no governo e quantas são crianças?

In [25]:
# Verificando a quantidade de entradas dos tipos de trabalho (work_type)
df.groupBy("work_type").count().show()

+-------------+-----+
|    work_type|count|
+-------------+-----+
| Never_worked|  177|
|Self-employed|14736|
|      Private|37806|
|     children| 6156|
|     Govt_job| 8260|
+-------------+-----+



In [16]:
# Criando uma tabela temporária
df.createOrReplaceTempView('table')

# Escrever consulta SQL
consulta_work_type = """
SELECT work_type, COUNT(*) AS stroke
FROM table
WHERE stroke = '1'
AND work_type NOT IN ('Never_worked')
GROUP BY work_type
"""

# Executar consulta SQL
resultado_work_type = spark.sql(consulta_work_type)

# Mostrar o resultado na tela
resultado_work_type.show()


+-------------+------+
|    work_type|stroke|
+-------------+------+
|Self-employed| 10807|
|      Private| 23711|
|     children|   520|
|     Govt_job|  5164|
+-------------+------+



In [28]:
# Visualização da quantidade de entradas dos generos participantes:
df.groupBy("gender").count().show()

+------+-----+
|gender|count|
+------+-----+
|Female|39530|
| Other|   11|
|  Male|27594|
+------+-----+



#### 5) Escreva uma consulta com spark.sql para determinar a proporção, por gênero, de participantes do estudo. A maioria dos participantes é:

In [37]:
# Consulta SQL para determinar a proporção por gênero
consulta_gender = """
SELECT gender,
       COUNT(*) AS total_subjects,
       ROUND (COUNT(*) / (SELECT COUNT(*) FROM table) * 100, 4) AS proportion
FROM table
GROUP BY gender
"""

# Executar consulta SQL
resultado_gender = spark.sql(consulta_gender)

# Mostrar o resultado na tela
resultado_gender.show()

+------+--------------+----------+
|gender|total_subjects|proportion|
+------+--------------+----------+
|Female|         39530|   58.8814|
| Other|            11|    0.0164|
|  Male|         27594|   41.1023|
+------+--------------+----------+



#### 6) Escreva uma consulta com spark.sql para determinar quem tem mais probabilidade de sofrer derrame: hipertensos ou não-hipertensos. Você pode escrever uma consulta para cada grupo. A partir das probabilidades que você obteve, você conclui que: 

In [49]:
# Visualizando totais de entradas das variáveis hypertension e stroke
df.groupBy('hypertension').count().show()
df.groupBy('stroke').count().show()

+------------+-----+
|hypertension|count|
+------------+-----+
|           1|11017|
|           0|56118|
+------------+-----+

+------+-----+
|stroke|count|
+------+-----+
|     1|40287|
|     0|26848|
+------+-----+



In [63]:
# Visualizando totais de entradas da variável hypertension apenas quando stroke é igual 1 (pacientes que tiveram derrame)

# Filtrando o dataframe
df_filtered = df.filter(df.stroke == 1)

# Visuzalizando o resultado
df_filtered.groupBy('hypertension').count().show()

+------------+-----+
|hypertension|count|
+------------+-----+
|           1| 8817|
|           0|31470|
+------------+-----+



In [25]:
# Criando uma tabela temporária
df.createOrReplaceTempView('table')

# Consulta SQL para calcular a probabilidade de sofrer um derrame com base na condição de hipertensão
consulta_hypertension_stroke = """
    SELECT
        AVG(CASE WHEN hypertension = 1 THEN stroke ELSE NULL END) AS prob_stroke_hypertension_1,
        AVG(CASE WHEN hypertension = 0 THEN stroke ELSE NULL END) AS prob_stroke_hypertension_0
    FROM
        table
"""

# Executar consulta SQL
resultado_hypertension_stroke = spark.sql(consulta_hypertension_stroke)


# Mostrar o resultado na tela
resultado_hypertension_stroke.show()


+--------------------------+--------------------------+
|prob_stroke_hypertension_1|prob_stroke_hypertension_0|
+--------------------------+--------------------------+
|        0.8003086139602432|        0.5607826365871913|
+--------------------------+--------------------------+



##### R: os pacientes  hipertensos (hypertension = 1 equivalente a SIM, possui histórico de hipertensão) apresentaram maior proabilidade de sofrerem derrame

#### 7) Escreva uma consulta com spark.sql que determine o número de pessoas que sofreram derrame por idade. Com qual idade o maior número de pessoas do conjunto de dados sofreu derrame?

In [8]:
# Consulta SQL para determinar a proporção por gênero
consulta_age = """
SELECT age, COUNT(*) AS stroke
FROM table
WHERE stroke = '1'
GROUP BY age
"""

# Executar consulta SQL
resultado_age = spark.sql(consulta_age)

# Ordenar o DataFrame em ordem descendente pela coluna 'stroke'
resultado_age_ordenado = resultado_age.orderBy("stroke", ascending=False)

# Mostrar o resultado na tela
resultado_age_ordenado.show()

+----+------+
| age|stroke|
+----+------+
|79.0|  2916|
|78.0|  2279|
|80.0|  1858|
|81.0|  1738|
|82.0|  1427|
|77.0|   994|
|74.0|   987|
|63.0|   942|
|76.0|   892|
|70.0|   881|
|66.0|   848|
|75.0|   809|
|67.0|   801|
|57.0|   775|
|73.0|   759|
|65.0|   716|
|72.0|   709|
|68.0|   688|
|69.0|   677|
|71.0|   667|
+----+------+
only showing top 20 rows



#### 8) Usando a API de dataframes, determine quantas pessoas sofreram derrames após os 50 anos.

In [14]:
# Importando funções do pyspark.sql
from pyspark.sql import functions as F

# Filtrando as pessoas com mais de 50 anos
resultado_age_filtrado = resultado_age_ordenado.filter(resultado_age_ordenado.age > 50)

# Somando o total dessas pessoas
resultado_age_50_mais = resultado_age_filtrado.agg(F.sum("stroke")).collect()[0][0]

# Printando resultado
print("Total de pessoas com derrame acima dos 50 anos: ", resultado_age_50_mais) 

Total de pessoas com derrame acima dos 50 anos:  28938


#### 9) Usando spark.sql, determine qual o nível médio de glicose para pessoas que, respectivamente, sofreram e não sofreram derrame.

In [8]:
# Registro temporário do DataFrame para poder executar consultas SQL
df.createOrReplaceTempView("table")

# Consulta SQL para calcular o nível médio de glicose para pessoas que sofreram ou não derrame
stroke_avg_glucose_level = spark.sql("""
    SELECT stroke,
           CONCAT(' ', ROUND(AVG(avg_glucose_level), 4)) AS avg_gucose_level
    FROM data
    GROUP BY stroke
""")

stroke_avg_glucose_level.show()


+------+----------------+
|stroke|avg_gucose_level|
+------+----------------+
|     1|        119.9531|
|     0|        103.6027|
+------+----------------+



#### 10) Qual é o BMI (IMC = índice de massa corpórea) médio de quem sofreu e não sofreu derrame?

In [9]:
# Registro temporário do DataFrame para poder executar consultas SQL
df.createOrReplaceTempView("table")

# Consulta SQL para calcular o nível médio de glicose para pessoas que sofreram ou não derrame
stroke_bmi = spark.sql("""
    SELECT stroke,
           CONCAT(' ', ROUND(AVG(bmi), 2)) AS avg_bmi
    FROM data
    GROUP BY stroke
""")

stroke_bmi.show()

+------+--------+
|stroke| avg_bmi|
+------+--------+
|     1| 29.9425|
|     0| 27.9897|
+------+--------+



#### 11) Crie um modelo de árvore de decisão que prevê a chance de derrame (stroke) a partir das variáveis contínuas/categóricas: idade, BMI, hipertensão, doença do coração, nível médio de glicose

In [3]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Determinando as coluans númericas para prever os derrames    
numeric_cols = ['age', 'bmi', 'hypertension', 'heart_disease', 'avg_glucose_level']
 
# Transformando o dataframe em um vetor com colunas categóricas   
vec_assembler = VectorAssembler(inputCols=numeric_cols, outputCol="features")

# Criando o objeto que será usado para treinar o modelo de Árvore de Decisão
dtc = DecisionTreeClassifier(labelCol='stroke', featuresCol='features')

# Criando o pipeline do modelo
pipeline = Pipeline(stages=[vec_assembler, dtc])

# Separando os dados em treinamento e teste
train_data, test_data = df.randomSplit([0.7, 0.3])

# Treinamento dos dados
pipeline_model = pipeline.fit(train_data)

# Execução da previsão
predictions_df = pipeline_model.transform(test_data)

# Avaliando a acurácia das previsões
evaluator = MulticlassClassificationEvaluator(metricName='accuracy', labelCol='stroke')
print(f"Acurácia: {evaluator.evaluate(predictions_df)}")


Acurácia: 0.6864829789358091


#### 12) Adicione ao modelo as variáveis categóricas: gênero e status de fumante. Use o conteúdo da aula interativa para lidar com as variáveis categóricas.  A acurácia (qualidade) do modelo aumentou para:

In [4]:
# Analisando os valores distintos das colunas que serão acrescentadas ao modelo
df.select('gender').distinct().show()
df.select('smoking_status').distinct().show()

+------+
|gender|
+------+
|Female|
| Other|
|  Male|
+------+

+---------------+
| smoking_status|
+---------------+
|         smokes|
|   never_smoked|
|formerly smoked|
+---------------+



In [23]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

# Determinando as coluans categóricas e numéricas para prever os derrames    
categorical_cols = ['gender', 'smoking_status']
numeric_cols = ['age', 'bmi', 'hypertension', 'heart_disease', 'avg_glucose_level']

# Convertendo valores categóricos string para numéricos
stringIndexer = StringIndexer(inputCols=categorical_cols, outputCols=[x + "Index" for x in categorical_cols]) 
oneHotEncoder = OneHotEncoder(inputCols=stringIndexer.getOutputCols(), outputCols=[x + "OHE" for x in categorical_cols])

# Agrupando todas as colunas
all_cols = [c + "OHE" for c in categorical_cols] + numeric_cols

In [24]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Transformando o dataframe em um vetor com colunas categóricas   
vec_assembler = VectorAssembler(inputCols=all_cols, outputCol="features")

# Criando o objeto que será usado para treinar o modelo de Árvore de Decisão
dtc = DecisionTreeClassifier(labelCol='stroke', featuresCol='features')

# Criando o pipeline do modelo
pipeline = Pipeline(stages=[stringIndexer, oneHotEncoder, vec_assembler, dtc])

# Separando os dados em treinamento e teste
train_data, test_data = df.randomSplit([0.7, 0.3])

# Treinamento dos dados
pipeline_model = pipeline.fit(train_data)

# Execução da previsão
predictions_df = pipeline_model.transform(test_data)

# Avaliando a acurácia das previsões
evaluator = MulticlassClassificationEvaluator(metricName='accuracy', labelCol='stroke')
print(f"Acurácia: {evaluator.evaluate(predictions_df)}")

Acurácia: 0.8386762675083843


### Podemos concluir que, as variáveis status de fumante (smoking_status) e genero (gender), apresentaram maior acurácia no modelo, melhorando o fit para previsão dos derrames.